# Acquire

In [1]:
# General Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Web Scraping Libraries
import urllib
import requests
from requests import get
from bs4 import BeautifulSoup

# Regex Library
import re

# Time-related Libraries
import time

# NLP Libraries
import unicodedata
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud

import MVP_Bojado

import warnings
warnings.filterwarnings("ignore")

## LinkedIn Job Postings

In [2]:
import requests
from bs4 import BeautifulSoup

client = requests.Session()

HOMEPAGE_URL = 'https://www.linkedin.com'
LOGIN_URL = "https://www.linkedin.com/uas/login-submit"

html = client.get(HOMEPAGE_URL).content

soup = BeautifulSoup(html, "html.parser")

csrf = soup.find('input')['value']

login_information = {
    'session_key':'user_name',
    'session_password':'password',
    'loginCsrfParam': csrf,
}

client.post(LOGIN_URL, data=login_information)

client.get('https://www.linkedin.com/jobs/search/?keywords=data%20science')

<Response [200]>

In [3]:
response = client.get('https://www.linkedin.com/jobs/search/?keywords=data%20science')

In [4]:
# Create a beautiful soup object of job postings in html format.
soup = BeautifulSoup(response.content, 'html.parser')

# Display the number of job postings collected.
len(soup.find_all('h3', class_="result-card__title job-result-card__title"))

0

## Job Title

In [5]:
for job in soup.find_all('h3', class_="result-card__title job-result-card__title"):
    print(job.text)

## Location

In [6]:
for location in soup.find_all('span', class_="job-result-card__location"):
    print(location.text)

## Company Name

In [7]:
for i in soup.find_all('h4', class_="result-card__subtitle job-result-card__subtitle"):
    print(i.text)

## Original Job Posting Date

In [8]:
for i in soup.find_all('time', class_="job-result-card__listdate"):
    print(i['datetime'])

## Job Post Age

In [9]:
for age in soup.find_all('time', class_="job-result-card__listdate"):
    print(age.text)

## Description

In [10]:
for i in soup.find_all('p'):
    print(i.text)

In [11]:
print(soup.prettify()[:500])

<!DOCTYPE html>
<html class="theme theme--mercado" lang="en">
 <head>
  <script type="application/javascript">
   !function(i,n){void 0!==i.addEventListener&&void 0!==i.hidden&&(n.liVisibilityChangeListener=function(){i.hidden&&(n.liHasWindowHidden=!0)},i.addEventListener("visibilitychange",n.liVisibilityChangeListener))}(document,window);
  </script>
  <title>
   LinkedIn
  </title>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta class="mercado-icons-s


# LinkedIn Job Posting Dataset